In [187]:
import pyranges as pr
import pyfaidx
import re
import textwrap

In [191]:
def write_chr(seq, ofile, chr_name, line_lim=None):
    with open(ofile, 'w') as ofile:
        ofile.write(f'>{chr_name}\n')
        if line_lim:
            wrapped_seq = textwrap.wrap(seq, width=line_lim)
            for seq in wrapped_seq:
                ofile.write(seq+'\n')
        else:
            ofile.write(str(seq))
        
def get_gene_seq(fa_file,
                gtf_file, 
                gene,
                ofile=None,
                slack=0):
        
    gtf_df = pr.read_gtf(gtf_file, as_df=True)

    gtf_df = gtf_df.loc[(gtf_df.gene_name==gene)&(gtf_df.Feature=='gene')]
    assert len(gtf_df.index) == 1

    # get start and end of gene
    start = min(gtf_df['Start'].values[0], gtf_df['End'].values[0])-slack
    end = max(gtf_df['Start'].values[0], gtf_df['End'].values[0])+slack
    ch = gtf_df['Chromosome'].values[0]
    print(ch)
    print(start)
    print(end)
    strand = gtf_df['Strand'].values[0]

    fa = pyfaidx.Fasta(fa_file)
    if strand == '+':
        gene_seq = fa[ch][start:end]
    else: 
        gene_seq = fa[ch][start:end].complement

    if ofile:
        write_chr(gene_seq.seq, ofile, gene)
    
    return gene_seq.seq

In [94]:
# mouse
gene = 'Clu'
gtf_file = 'ref/annot.gtf'
fa_file = 'ref/genome.fa'
ofile = 'ref/clu.fa'
seq = get_gene_seq(fa_file,
             gtf_file, 
             gene,
             ofile,
             slack=1500)

chr14
65966982
65983047


In [184]:
def replace_seq(mod_fa,
                seq):
    """
    Replace sequence in seq with sequence in mod_fa. Use where
    anchor_5 and anchor_3 are within seq to determine where
    to replace.
    
    Parameters:
        mod_fa (str): Path to fasta file with sequence to insert
        seq (str): Sequence to add mod_fa into
    """
    
    # get anchors at 5' and 3' ends of sequence
    fa = pyfaidx.Fasta(modified_fa)
    mod_seq = fa[list(fa.keys())[0]][0:-1].seq
    
    anchor_5 = mod_seq[:20]
    anchor_3 = mod_seq[-20:]
    
    print(anchor_5)
    print(anchor_3)
    
    start_inds = [(m.start(), m.end()) for m in re.finditer(anchor_5, seq, re.IGNORECASE)]
    print(start_inds)
    assert len(start_inds) == 1

    end_inds = [(m.start(), m.end()) for m in re.finditer(anchor_3, seq, re.IGNORECASE)]
    print(end_inds)
    assert len(end_inds) == 1

    print(seq[start_inds[0][0]:start_inds[0][1]])
    assert seq[start_inds[0][0]:start_inds[0][1]].lower() == anchor_5.lower()
    assert seq[end_inds[0][0]:end_inds[0][1]].lower() == anchor_3.lower()

    # replace_seq = seq[start_inds[0][1]:end_inds[0][0]] # this will replace excluding anchors
    replace_seq = seq[start_inds[0][0]:end_inds[0][1]] # this will replace inluding anchors
    with open('ref/replace_seq.fa', 'w') as ofile:
        ofile.write(replace_seq)
        
    assert replace_seq[:20].lower() == anchor_5.lower()
    assert replace_seq[-20:].lower() == anchor_3.lower()
    
    # print(replace_seq[0])
    # print(replace_seq[-1])

    inds = [(m.start(), m.end()) for m in re.finditer(replace_seq, seq)]
    print(inds)
    assert len(inds) == 1  
    
    print(seq.find(replace_seq))
    new_len = len(seq)-len(replace_seq)+len(mod_seq)
    new_seq = seq.replace(replace_seq, mod_seq)
    assert len(new_seq) == new_len
    return new_seq

In [185]:
gene = 'Clu'
gtf_file = 'ref/annot.gtf'
fa_file = 'ref/genome.fa'
seq = get_gene_seq(fa_file,
             gtf_file, 
             gene,
             ofile = 'ref/clu.fa',
             slack=3000)

In [192]:
mod_fa = 'ref/Clu-2kb_info/ID16848_consensus_040622.fas'
ofile = 'ref/hClu.fa'
new_seq = replace_seq(mod_fa,
                      seq)
write_chr(new_seq, ofile, 'hClu', line_lim=80)

TGGAGTGCCTACTTGGCTAG
ATGTCGTATGtgtaaggcca
[(14987, 15007)]
[(17623, 17643)]
TGGAGTGCCTACTTGGCTAG
[(14987, 17643)]
14987


In [98]:
# print(65963482<65982941)
# print(65986547>65983125)

In [96]:
# # human
# gene = 'CLU'
# gtf_file = 'ref/human_annot.gtf'
# fa_file = 'ref/human_genome.fa'
# ofile = 'ref/human_clu.fa'
# get_gene_seq(fa_file,
#              gtf_file, 
#              gene,
#              ofile)

In [124]:
modified_fa = 'ref/Clu-2kb_info/ID16848_consensus_040622.fas'
fa = pyfaidx.Fasta(modified_fa)
mod_seq = fa[list(fa.keys())[0]][0:-1].seq

In [97]:
mouse_5_anchor='ATAACCCGCAGATCCAT' # hand checked if these exist in the Clu mm10
mouse_3_anchor='TGTACAGGCACTCAA'

In [125]:
start_inds = [(m.start(), m.end()) for m in re.finditer(mouse_5_anchor, seq)]
assert len(start_inds) == 1

end_inds = [(m.start(), m.end()) for m in re.finditer(mouse_3_anchor, seq)]
assert len(end_inds) == 1

assert seq[start_inds[0][0]:start_inds[0][1]] == mouse_5_anchor
assert seq[end_inds[0][0]:end_inds[0][1]] == mouse_3_anchor

replace_seq = seq[start_inds[0][1]:end_inds[0][0]]
# print(replace_seq[0])
# print(replace_seq[-1])

inds = [(m.start(), m.end()) for m in re.finditer(replace_seq, seq)]
assert len(inds) == 1

# seq = seq.replace(replace_seq, mod_seq)


In [73]:
len(seq)

19065

In [65]:
# temp_ind = seq.find(mouse_5_anchor)
# start_ind = seq.find(mouse_5_anchor)+len(mouse_5_anchor)
# print(seq[temp_ind:start_ind+1])

# end_ind = seq.find(mouse_3_anchor)
# temp_ind = seq.find(mouse_3_anchor)+len(mouse_3_anchor)
# # print(seq[end_ind:temp_ind])

# # replace_seq = seq[start_ind+2:end_ind-1]
# # # replace_seq = seq[start_ind:end_ind]

# # print(replace_seq[0])
# # print(replace_seq[1])

ATAACCCGCAGATCCATA


In [60]:
end_ind

-1

In [36]:
# # mouse


# gtf_df = pr.read_gtf(gtf_file, as_df=True)

# gtf_df = gtf_df.loc[(gtf_df.gene_name==gene)&(gtf_df.Feature=='gene')]
# assert len(gtf_df.index) == 1

# # get start and end of gene
# start = min(gtf_df['Start'].values[0], gtf_df['End'].values[0])
# end = max(gtf_df['Start'].values[0], gtf_df['End'].values[0])
# ch = gtf_df['Chromosome'].values[0]

# fa = pyfaidx.Fasta(fa_file)
# gene_seq = fa[ch][start:end]

# with open(mouse_ofile, 'w') as ofile:
#     ofile.write('>clu\n')
#     ofile.write(str(gene_seq.seq))

In [37]:
# # human
# gene = 'CLU'
# gtf_file = 'ref/human_annot.gtf'
# fa_file = 'ref/human_genome.fa'
# human_ofile = 'ref/human_clu.fa'

# gtf_df = pr.read_gtf(gtf_file, as_df=True)

# gtf_df = gtf_df.loc[(gtf_df.gene_name==gene)&(gtf_df.Feature=='gene')]
# assert len(gtf_df.index) == 1

# # get start and end of gene
# start = min(gtf_df['Start'].values[0], gtf_df['End'].values[0])
# end = max(gtf_df['Start'].values[0], gtf_df['End'].values[0])
# ch = gtf_df['Chromosome'].values[0]
# strand = gtf_df['Strand'].values[0]

# fa = pyfaidx.Fasta(fa_file)
# if strand == '+':
#     gene_seq = fa[ch][start:end]
# else: 
#     gene_seq = fa[ch][start:end].complement

# with open(human_ofile, 'w') as ofile:
#     ofile.write('>clu\n')
#     ofile.write(str(gene_seq.seq))

In [30]:
gtf_df

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
1133585,chr8,HAVANA,gene,27596916,27615031,.,-,.,ENSG00000120885.21,protein_coding,...,NaN,NaN,NaN,ncRNA_host,NaN,NaN,NaN,NaN,NaN,NaN
